In [60]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)

rows_list = []

def safe_click(element):
    retries = 3
    for _ in range(retries):
        try:
            element.click()
            return
        except Exception as e:
            print(f"Error clicking element: {e}. Retrying...")
            time.sleep(2)
    raise Exception(f"Failed to click element after {retries} retries")

def get_vehicle_list():
    try:
        main_div_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
        )

        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")

        for div_index in range(len(div_elements)):
            driver.refresh()
            time.sleep(3)
            try:
                main_div_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                )
                div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                div_element = div_elements[div_index]
                a_elements = div_element.find_elements(By.TAG_NAME, "a")

                for a_index in range(len(a_elements)):
                    driver.refresh()
                    time.sleep(3)
                    try:
                        main_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )
                        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                        div_element = div_elements[div_index]
                        a_elements = div_element.find_elements(By.TAG_NAME, "a")
                        a_element = a_elements[a_index]

                        vehicle_name = a_element.text
                        vehicle_url = driver.current_url

                        # Wait until the element is clickable and scroll to it
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(a_element))
                        driver.execute_script("arguments[0].scrollIntoView(true);", a_element)
                        driver.execute_script("arguments[0].click();", a_element)

#                         safe_click(a_element)

                        # Get current page URL
                        current_url = driver.current_url
                        print(f"Current URL: {current_url}")

                        # Re-locate the markets_div_element after navigation
                        markets_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )

                        markets = markets_div_element.find_elements(By.TAG_NAME, "div")

                        for market_index in range(len(markets)):
                            driver.refresh()
                            time.sleep(3)
                            try:
                                markets_div_element = WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                )
                                markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                                market = markets[market_index]

                                market_tags = market.find_elements(By.CLASS_NAME, "market__title")
                                if not market_tags:
                                    print(f"Market title not found at div {div_index}, a {a_index}, market {market_index}")
                                    continue
                                market_tag = market_tags[0]
                                market_name = market_tag.text

                                chassi_models_tag = market.find_elements(By.TAG_NAME, "a")

                                for chassi_index in range(len(chassi_models_tag)):
                                    try:
                                        markets_div_element = WebDriverWait(driver, 10).until(
                                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                        )
                                        markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                                        market = markets[market_index]
                                        
                                        chassi_models_tag = market.find_elements(By.TAG_NAME, "a")
                                        a1_element = chassi_models_tag[chassi_index]

                                        chassi_model_name = a1_element.text

                                        # Wait until the element is clickable and scroll to it
                                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(a1_element))
                                        driver.execute_script("arguments[0].scrollIntoView(true);", a1_element)
                                        driver.execute_script("arguments[0].click();", a1_element)
#                                         safe_click(a1_element)

                                        # Get current page URL
                                        current_url = driver.current_url
                                        print(f"Current URL: {current_url}")

                                        # Re-locate the table_element after navigation
                                        table_element = WebDriverWait(driver, 10).until(
                                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[2]/table/tbody'))
                                        )

                                        table_rows = table_element.find_elements(By.CLASS_NAME, "epcVariations__row")

                                        for table_row in table_rows:
                                            table_data = table_row.find_elements(By.TAG_NAME, "td")

                                            model_tag = table_data[0].find_element(By.TAG_NAME, "a")
                                            model_name = model_tag.text

                                            engine_tag = table_data[1].find_element(By.TAG_NAME, "span")
                                            engine_name = engine_tag.text
                                            engine_name_code = engine_tag.get_attribute('data-content')

                                            prod_period_name = table_data[2].text

                                            # Handle potential empty <td> for body
                                            try:
                                                body_tag = table_data[3].find_element(By.TAG_NAME, "span")
                                                body_name = body_tag.text
                                                body_name_code = body_tag.get_attribute('data-content')
                                            except:
                                                body_name = ""
                                                body_name_code = ""

                                            # Handle potential empty <td> for grade
                                            try:
                                                grade_tag = table_data[4].find_element(By.TAG_NAME, "span")
                                                grade_name = grade_tag.text
                                                grade_name_code = grade_tag.get_attribute('data-content')
                                            except:
                                                grade_name = ""
                                                grade_name_code = ""

                                            # Handle potential empty <td> for transmission
                                            try:
                                                transmission_tag = table_data[5].find_element(By.TAG_NAME, "span")
                                                transm_name = transmission_tag.text
                                                transm_name_code = transmission_tag.get_attribute('data-content')
                                            except:
                                                transm_name = ""
                                                transm_name_code = ""

                                            options = table_data[6].find_elements(By.TAG_NAME, "span")
                                            option_names = {option.text: option.get_attribute('data-content') for option in options}

                                            new_row = {
                                                "Brand": "Toyota",  # Replace with actual brand if needed
                                                "Vehicle": vehicle_name,
                                                "Market": market_name,
                                                "Page_URL": vehicle_url,
                                                "Chassi_model": chassi_model_name,
                                                "Model": model_name,
                                                "Engine": engine_name,
                                                "Engine_code": engine_name_code,
                                                "Prod period": prod_period_name,
                                                "Body": body_name,
                                                "Body_code": body_name_code,
                                                "Grade": grade_name,
                                                "Grade_code": grade_name_code,
                                                "Transm": transm_name,
                                                "Transm_code": transm_name_code,
                                                "Options": option_names
                                            }

                                            rows_list.append(new_row)
                                            
                                    
                                    except Exception as e:
                                        print(f"Error processing chassi model at div {div_index}, a {a_index}, market {market_index}, chassi {chassi_index}: {e}")
                                    finally:
                                        driver.back()
                                        time.sleep(2)
                                        
                            except Exception as e:
                                print(f"Error processing market at div {div_index}, a {a_index}, market {market_index}: {e}")
                                # Re-locate the elements to avoid stale reference
                                markets_div_element = WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                )
                                markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                            finally:
                                if chassi_index != (len(chassi_models_tag)-1):
                                    print(chassi_index, len(chassi_models_tag)-1)
                                    driver.back()
                                time.sleep(2)
                                
                    except Exception as e:
                        print(f"Error processing vehicle at div {div_index}, a {a_index}: {e}")
                        # Re-locate the elements to avoid stale reference
                        main_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )
                        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                    finally:
                        driver.back()
                        time.sleep(2)
            except Exception as e:
                print(f"Error processing div {div_index}: {e}")
                main_div_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                )
                div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
            finally:
#                 driver.back()
                time.sleep(2)
    except Exception as e:
        print(f"Error in main process: {e}")

if __name__ == "__main__":
    driver.get("https://www.amayama.com/en/catalogs/toyota")
    get_vehicle_list()
    driver.quit()

    # Convert the list of dictionaries to a DataFrame and save as CSV
    try:
        output_file = "vehicle_data.csv"
        df = pd.DataFrame(rows_list)
        df.to_csv(output_file, index=False)
        print(df)
    except PermissionError as pe:
        print(f"PermissionError: {pe}")
    except Exception as e:
        print(f"An error occurred while saving the CSV file: {e}")


Current URL: https://www.amayama.com/en/catalogs/toyota/4runner
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/GRN210L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/GRN215L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/GRN280L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/GRN285L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/LN51L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/LN56L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/LN65L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/LN66L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/RN101L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/4runner/RN101L?ref

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/allion/ZZT245?ref=ae
Market title not found at div 1, a 1, market 1
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/allion/MZEA12L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/allion/MZEH16L?ref=ae
Market title not found at div 1, a 1, market 3
Current URL: https://www.amayama.com/en/catalogs/toyota/alphard
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/alphard/AAHH40W?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/alphard/AAHH45W?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/alphard/AGH30W?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/alphard/AGH35W?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/alphard/AGH40W?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-j

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/ADE150L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/ADE150R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/ADE150R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/ADE157L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/ADE157R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/ADE186L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/NDE150L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/NDE150L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/NDE150R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/auris/NDE150R?ref=ae
Current URL: https://www.amayama.com/en/

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/AT221R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/AZT220L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/AZT220R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/AZT250L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/AZT250R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/AZT251L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/AZT251R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/CDT220L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/CDT220R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/avensis/CDT250L?ref=ae
Current URL: https://

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/chr/ZYX10?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/chr/ZYX11?ref=ae
Market title not found at div 3, a 1, market 1
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/chr/ZGX10L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-usa/chr/ZGX10L?ref=ae
Market title not found at div 3, a 1, market 3
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/chr/KPX10L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/chr/MAXA10L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/chr/MAXH10L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/chr/MAXH20R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/chr/MAXH25R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-ge

...

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/ASV70R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/ASV71L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/ASV71L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/ASV71R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/AVV50L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/AVV50L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/AVV50R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/AXVA70L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/AXVA70L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/camry/AXVA70L?ref=ae
Current URL: https://www.amayama.com/

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/MCV20R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/MCV30L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/MCV30R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/MXVA71L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/SV10L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/SV10R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/SV11L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/SV11R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/SV20L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-europe/camry/SV20LG?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalo

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima/TCR20W?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima/TCR21W?ref=ae
Market title not found at div 5, a 2, market 1
Current URL: https://www.amayama.com/en/catalogs/toyota/estima-emina
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima_emina/CXR10G?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima_emina/CXR11G?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima_emina/CXR20G?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima_emina/CXR21G?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima_emina/TCR10G?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima_emina/TCR11G?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/estima_emina/T

Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/granvia/GRH304R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/granvia/GRH320L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/granvia/GRH320R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/granvia/GRH321L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/granvia/GRH321R?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/granvia/GRH322L?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-general/granvia/GRH322R?ref=ae
Market title not found at div 7, a 3, market 3
Current URL: https://www.amayama.com/en/catalogs/toyota/gt-86
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/86/GRMN86?ref=ae
Current URL: https://www.amayama.com/en/genuine-catalogs/epc/toyota-japan/86/ZN6?ref=ae
Current URL: https:

Current URL: https://www.amayama.com/ban.html
Error processing chassi model at div 7, a 4, market 6, chassi 14: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF71EC41F52+60322]
	(No symbol) [0x00007FF71EBBCEC9]
	(No symbol) [0x00007FF71EA77EBA]
	(No symbol) [0x00007FF71EA4D5A5]
	(No symbol) [0x00007FF71EAF36B7]
	(No symbol) [0x00007FF71EB0B331]
	(No symbol) [0x00007FF71EAEBFC3]
	(No symbol) [0x00007FF71EAB9617]
	(No symbol) [0x00007FF71EABA211]
	GetHandleVerifier [0x00007FF71EF594AD+3301629]
	GetHandleVerifier [0x00007FF71EFA36D3+3605283]
	GetHandleVerifier [0x00007FF71EF99450+3563680]
	GetHandleVerifier [0x00007FF71ECF4326+790390]
	(No symbol) [0x00007FF71EBC750F]
	(No symbol) [0x00007FF71EBC3404]
	(No symbol) [0x00007FF71EBC3592]
	(No symbol) [0x00007FF71EBB2F9F]
	BaseThreadInitThunk [0x00007FF8FA777344+20]
	RtlUserThreadStart [0x00007FF8FAB5CC91+33]

Error pr

In [46]:
for i in range(5):
    print(i)

0
1
2
3
4


In [62]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)

rows_list = []

def safe_click(element):
    retries = 3
    for _ in range(retries):
        try:
            element.click()
            return
        except Exception as e:
            print(f"Error clicking element: {e}. Retrying...")
            time.sleep(2)
    raise Exception(f"Failed to click element after {retries} retries")

def get_vehicle_list():
    try:
        main_div_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
        )

        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")

        for div_index in range(7, len(div_elements)):
            driver.refresh()
            time.sleep(3)
            try:
                main_div_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                )
                div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                div_element = div_elements[div_index]
                a_elements = div_element.find_elements(By.TAG_NAME, "a")

                for a_index in range(4, len(a_elements)):
                    driver.refresh()
                    time.sleep(3)
                    try:
                        main_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )
                        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                        div_element = div_elements[div_index]
                        a_elements = div_element.find_elements(By.TAG_NAME, "a")
                        a_element = a_elements[a_index]

                        vehicle_name = a_element.text
                        vehicle_url = driver.current_url

                        # Wait until the element is clickable and scroll to it
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(a_element))
                        driver.execute_script("arguments[0].scrollIntoView(true);", a_element)
                        driver.execute_script("arguments[0].click();", a_element)

#                         safe_click(a_element)

                        # Get current page URL
                        current_url = driver.current_url
                        print(f"Current URL: {current_url}")

                        # Re-locate the markets_div_element after navigation
                        markets_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )

                        markets = markets_div_element.find_elements(By.TAG_NAME, "div")

                        for market_index in range(6, len(markets)):
                            driver.refresh()
                            time.sleep(3)
                            try:
                                markets_div_element = WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                )
                                markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                                market = markets[market_index]

                                market_tags = market.find_elements(By.CLASS_NAME, "market__title")
                                if not market_tags:
                                    print(f"Market title not found at div {div_index}, a {a_index}, market {market_index}")
                                    continue
                                market_tag = market_tags[0]
                                market_name = market_tag.text

                                chassi_models_tag = market.find_elements(By.TAG_NAME, "a")

                                for chassi_index in range(8, len(chassi_models_tag)):
                                    try:
                                        markets_div_element = WebDriverWait(driver, 10).until(
                                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                        )
                                        markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                                        market = markets[market_index]
                                        
                                        chassi_models_tag = market.find_elements(By.TAG_NAME, "a")
                                        a1_element = chassi_models_tag[chassi_index]

                                        chassi_model_name = a1_element.text

                                        # Wait until the element is clickable and scroll to it
                                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(a1_element))
                                        driver.execute_script("arguments[0].scrollIntoView(true);", a1_element)
                                        driver.execute_script("arguments[0].click();", a1_element)
#                                         safe_click(a1_element)

                                        # Get current page URL
                                        current_url = driver.current_url
                                        print(f"Current URL: {current_url}")

                                        # Re-locate the table_element after navigation
                                        table_element = WebDriverWait(driver, 10).until(
                                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[2]/table/tbody'))
                                        )

                                        table_rows = table_element.find_elements(By.CLASS_NAME, "epcVariations__row")

                                        for table_row in table_rows:
                                            table_data = table_row.find_elements(By.TAG_NAME, "td")

                                            model_tag = table_data[0].find_element(By.TAG_NAME, "a")
                                            model_name = model_tag.text

                                            engine_tag = table_data[1].find_element(By.TAG_NAME, "span")
                                            engine_name = engine_tag.text
                                            engine_name_code = engine_tag.get_attribute('data-content')

                                            prod_period_name = table_data[2].text

                                            # Handle potential empty <td> for body
                                            try:
                                                body_tag = table_data[3].find_element(By.TAG_NAME, "span")
                                                body_name = body_tag.text
                                                body_name_code = body_tag.get_attribute('data-content')
                                            except:
                                                body_name = ""
                                                body_name_code = ""

                                            # Handle potential empty <td> for grade
                                            try:
                                                grade_tag = table_data[4].find_element(By.TAG_NAME, "span")
                                                grade_name = grade_tag.text
                                                grade_name_code = grade_tag.get_attribute('data-content')
                                            except:
                                                grade_name = ""
                                                grade_name_code = ""

                                            # Handle potential empty <td> for transmission
                                            try:
                                                transmission_tag = table_data[5].find_element(By.TAG_NAME, "span")
                                                transm_name = transmission_tag.text
                                                transm_name_code = transmission_tag.get_attribute('data-content')
                                            except:
                                                transm_name = ""
                                                transm_name_code = ""

                                            options = table_data[6].find_elements(By.TAG_NAME, "span")
                                            option_names = {option.text: option.get_attribute('data-content') for option in options}

                                            new_row = {
                                                "Brand": "Toyota",  # Replace with actual brand if needed
                                                "Vehicle": vehicle_name,
                                                "Market": market_name,
                                                "Page_URL": vehicle_url,
                                                "Chassi_model": chassi_model_name,
                                                "Model": model_name,
                                                "Engine": engine_name,
                                                "Engine_code": engine_name_code,
                                                "Prod period": prod_period_name,
                                                "Body": body_name,
                                                "Body_code": body_name_code,
                                                "Grade": grade_name,
                                                "Grade_code": grade_name_code,
                                                "Transm": transm_name,
                                                "Transm_code": transm_name_code,
                                                "Options": option_names
                                            }

                                            rows_list.append(new_row)
                                            
                                    
                                    except Exception as e:
                                        print(f"Error processing chassi model at div {div_index}, a {a_index}, market {market_index}, chassi {chassi_index}: {e}")
                                    finally:
                                        driver.back()
                                        time.sleep(2)
                                        
                            except Exception as e:
                                print(f"Error processing market at div {div_index}, a {a_index}, market {market_index}: {e}")
                                # Re-locate the elements to avoid stale reference
                                markets_div_element = WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                )
                                markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                            finally:
                                if chassi_index != (len(chassi_models_tag)-1):
                                    print(chassi_index, len(chassi_models_tag)-1)
                                    driver.back()
                                time.sleep(2)
                                
                    except Exception as e:
                        print(f"Error processing vehicle at div {div_index}, a {a_index}: {e}")
                        # Re-locate the elements to avoid stale reference
                        main_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )
                        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                    finally:
                        driver.back()
                        time.sleep(2)
            except Exception as e:
                print(f"Error processing div {div_index}: {e}")
                main_div_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                )
                div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
            finally:
#                 driver.back()
                time.sleep(2)
    except Exception as e:
        print(f"Error in main process: {e}")

if __name__ == "__main__":
    driver.get("https://www.amayama.com/en/catalogs/toyota")
    get_vehicle_list()
    driver.quit()

    # Convert the list of dictionaries to a DataFrame and save as CSV
    try:
        output_file = "vehicle_data.csv"
        df = pd.DataFrame(rows_list)
        df.to_csv(output_file, index=False)
        print(df)
    except PermissionError as pe:
        print(f"PermissionError: {pe}")
    except Exception as e:
        print(f"An error occurred while saving the CSV file: {e}")


Error in main process: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF71EC41F52+60322]
	(No symbol) [0x00007FF71EBBCEC9]
	(No symbol) [0x00007FF71EA77EBA]
	(No symbol) [0x00007FF71EA4D5A5]
	(No symbol) [0x00007FF71EAF36B7]
	(No symbol) [0x00007FF71EB0B331]
	(No symbol) [0x00007FF71EAEBFC3]
	(No symbol) [0x00007FF71EAB9617]
	(No symbol) [0x00007FF71EABA211]
	GetHandleVerifier [0x00007FF71EF594AD+3301629]
	GetHandleVerifier [0x00007FF71EFA36D3+3605283]
	GetHandleVerifier [0x00007FF71EF99450+3563680]
	GetHandleVerifier [0x00007FF71ECF4326+790390]
	(No symbol) [0x00007FF71EBC750F]
	(No symbol) [0x00007FF71EBC3404]
	(No symbol) [0x00007FF71EBC3592]
	(No symbol) [0x00007FF71EBB2F9F]
	BaseThreadInitThunk [0x00007FF8FA777344+20]
	RtlUserThreadStart [0x00007FF8FAB5CC91+33]

Empty DataFrame
Columns: []
Index: []


In [ ]:
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)

rows_list = []
BAN_URL = "https://www.amayama.com/ban.html"
PROGRESS_FILE = "progress.json"

def wait_if_banned(driver):
    if driver.current_url == BAN_URL:
        print("Bot detected and banned. Waiting for a few minutes before retrying...")
        time.sleep(300)  # Wait for 5 minutes

def safe_click(element):
    retries = 3
    for _ in range(retries):
        try:
            element.click()
            return
        except Exception as e:
            print(f"Error clicking element: {e}. Retrying...")
            time.sleep(2)
    raise Exception(f"Failed to click element after {retries} retries")

def save_progress(progress):
    with open(PROGRESS_FILE, 'w') as f:
        json.dump(progress, f)

def load_progress():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as f:
            return json.load(f)
    return {"div_index": 0, "a_index": 0, "market_index": 0, "chassi_index": 0}

def get_vehicle_list():
    progress = load_progress()
    div_index_start = progress["div_index"]
    a_index_start = progress["a_index"]
    market_index_start = progress["market_index"]
    chassi_index_start = progress["chassi_index"]

    try:
        main_div_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
        )

        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")

        for div_index in range(div_index_start, len(div_elements)):
            driver.refresh()
            time.sleep(3)
            wait_if_banned(driver)
            try:
                main_div_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                )
                div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                div_element = div_elements[div_index]
                a_elements = div_element.find_elements(By.TAG_NAME, "a")

                for a_index in range(a_index_start, len(a_elements)):
                    driver.refresh()
                    time.sleep(3)
                    wait_if_banned(driver)
                    try:
                        main_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )
                        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                        div_element = div_elements[div_index]
                        a_elements = div_element.find_elements(By.TAG_NAME, "a")
                        a_element = a_elements[a_index]

                        vehicle_name = a_element.text
                        vehicle_url = driver.current_url

                        # Wait until the element is clickable and scroll to it
                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(a_element))
                        driver.execute_script("arguments[0].scrollIntoView(true);", a_element)
                        driver.execute_script("arguments[0].click();", a_element)

                        # Get current page URL
                        current_url = driver.current_url
                        print(f"Current URL: {current_url}")

                        wait_if_banned(driver)

                        # Re-locate the markets_div_element after navigation
                        markets_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )

                        markets = markets_div_element.find_elements(By.TAG_NAME, "div")

                        for market_index in range(market_index_start, len(markets)):
                            driver.refresh()
                            time.sleep(3)
                            wait_if_banned(driver)
                            try:
                                markets_div_element = WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                )
                                markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                                market = markets[market_index]

                                market_tags = market.find_elements(By.CLASS_NAME, "market__title")
                                if not market_tags:
                                    print(f"Market title not found at div {div_index}, a {a_index}, market {market_index}")
                                    continue
                                market_tag = market_tags[0]
                                market_name = market_tag.text

                                chassi_models_tag = market.find_elements(By.TAG_NAME, "a")

                                for chassi_index in range(chassi_index_start, len(chassi_models_tag)):
                                    try:
                                        markets_div_element = WebDriverWait(driver, 10).until(
                                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                        )
                                        markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                                        market = markets[market_index]
                                        
                                        chassi_models_tag = market.find_elements(By.TAG_NAME, "a")
                                        a1_element = chassi_models_tag[chassi_index]

                                        chassi_model_name = a1_element.text

                                        # Wait until the element is clickable and scroll to it
                                        WebDriverWait(driver, 10).until(EC.element_to_be_clickable(a1_element))
                                        driver.execute_script("arguments[0].scrollIntoView(true);", a1_element)
                                        driver.execute_script("arguments[0].click();", a1_element)

                                        # Get current page URL
                                        current_url = driver.current_url
                                        print(f"Current URL: {current_url}")

                                        wait_if_banned(driver)

                                        # Re-locate the table_element after navigation
                                        table_element = WebDriverWait(driver, 10).until(
                                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div[2]/table/tbody'))
                                        )

                                        table_rows = table_element.find_elements(By.CLASS_NAME, "epcVariations__row")

                                        for table_row in table_rows:
                                            table_data = table_row.find_elements(By.TAG_NAME, "td")

                                            model_tag = table_data[0].find_element(By.TAG_NAME, "a")
                                            model_name = model_tag.text

                                            engine_tag = table_data[1].find_element(By.TAG_NAME, "span")
                                            engine_name = engine_tag.text
                                            engine_name_code = engine_tag.get_attribute('data-content')

                                            prod_period_name = table_data[2].text

                                            # Handle potential empty <td> for body
                                            try:
                                                body_tag = table_data[3].find_element(By.TAG_NAME, "span")
                                                body_name = body_tag.text
                                                body_name_code = body_tag.get_attribute('data-content')
                                            except:
                                                body_name = ""
                                                body_name_code = ""

                                            # Handle potential empty <td> for grade
                                            try:
                                                grade_tag = table_data[4].find_element(By.TAG_NAME, "span")
                                                grade_name = grade_tag.text
                                                grade_name_code = grade_tag.get_attribute('data-content')
                                            except:
                                                grade_name = ""
                                                grade_name_code = ""

                                            # Handle potential empty <td> for transmission
                                            try:
                                                transmission_tag = table_data[5].find_element(By.TAG_NAME, "span")
                                                transm_name = transmission_tag.text
                                                transm_name_code = transmission_tag.get_attribute('data-content')
                                            except:
                                                transm_name = ""
                                                transm_name_code = ""

                                            options = table_data[6].find_elements(By.TAG_NAME, "span")
                                            option_names = {option.text: option.get_attribute('data-content') for option in options}

                                            new_row = {
                                                "Brand": "Toyota",  # Replace with actual brand if needed
                                                "Vehicle": vehicle_name,
                                                "Market": market_name,
                                                "Page_URL": vehicle_url,
                                                "Chassi_model": chassi_model_name,
                                                "Model": model_name,
                                                "Engine": engine_name,
                                                "Engine_code": engine_name_code,
                                                "Prod period": prod_period_name,
                                                "Body": body_name,
                                                "Body_code": body_name_code,
                                                "Grade": grade_name,
                                                "Grade_code": grade_name_code,
                                                "Transm": transm_name,
                                                "Transm_code": transm_name_code,
                                                "Options": option_names
                                            }

                                            rows_list.append(new_row)

                                    except Exception as e:
                                        print(f"Error processing chassi model at div {div_index}, a {a_index}, market {market_index}, chassi {chassi_index}: {e}")
                                    finally:
                                        driver.back()
                                        time.sleep(2)
                                    save_progress({"div_index": div_index, "a_index": a_index, "market_index": market_index, "chassi_index": chassi_index})
                                        
                            except Exception as e:
                                print(f"Error processing market at div {div_index}, a {a_index}, market {market_index}: {e}")
                                # Re-locate the elements to avoid stale reference
                                markets_div_element = WebDriverWait(driver, 10).until(
                                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                                )
                                markets = markets_div_element.find_elements(By.TAG_NAME, "div")
                            finally:
                                if chassi_index != (len(chassi_models_tag) - 1):
                                    print(chassi_index, len(chassi_models_tag) - 1)
                                    driver.back()
                                time.sleep(2)
                                save_progress({"div_index": div_index, "a_index": a_index, "market_index": market_index, "chassi_index": chassi_index})
                                
                    except Exception as e:
                        print(f"Error processing vehicle at div {div_index}, a {a_index}: {e}")
                        # Re-locate the elements to avoid stale reference
                        main_div_element = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                        )
                        div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
                    finally:
                        driver.back()
                        time.sleep(2)
                        save_progress({"div_index": div_index, "a_index": a_index, "market_index": market_index, "chassi_index": chassi_index})
            except Exception as e:
                print(f"Error processing div {div_index}: {e}")
                main_div_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div/div[2]'))
                )
                div_elements = main_div_element.find_elements(By.TAG_NAME, "div")
            finally:
                time.sleep(2)
                save_progress({"div_index": div_index, "a_index": a_index, "market_index": market_index, "chassi_index": chassi_index})
    except Exception as e:
        print(f"Error in main process: {e}")

if __name__ == "__main__":
    driver.get("https://www.amayama.com/en/catalogs/toyota")
    get_vehicle_list()
    driver.quit()

    # Convert the list of dictionaries to a DataFrame and save as CSV
    try:
        output_file = "vehicle_data.csv"
        df = pd.DataFrame(rows_list)
        df.to_csv(output_file, index=False)
        print(df)
    except PermissionError as pe:
        print(f"PermissionError: {pe}")
    except Exception as e:
        print(f"An error occurred while saving the CSV file: {e}")


In [1]:
import pandas as pd

In [11]:
df1 = pd.read_csv("vehicle_data.csv")
df1.shape

(5333, 16)

In [12]:
df2 = pd.read_csv("vehicle_data_2.csv")
df2.shape

(580, 16)

In [13]:
df3 = pd.read_csv("vehicle_data_3.csv")
df3.shape

(71, 16)

In [14]:
df4 = pd.read_csv("toyace.csv")
df4.shape

(136, 16)

In [15]:
df5 = pd.read_csv("town_ace.csv")
df5.shape

(1145, 16)

In [16]:
df6 = pd.read_csv("vitz.csv")
df6.shape

(49, 16)

In [17]:
df7 = pd.read_csv("voxy.csv")
df7.shape

(166, 16)

In [18]:
df8 = pd.read_csv("last.csv")
df8.shape

(1996, 16)

In [10]:
def concatenate_dataframes_rowwise(dfs: list) -> pd.DataFrame:
    """Concatenate a list of DataFrames row-wise."""
    concatenated_df = pd.concat(dfs, axis=0, ignore_index=True)
    return concatenated_df

In [19]:
data = concatenate_dataframes_rowwise([df1,df2,df3,df4,df5,df6,df7,df8])

In [20]:
data.shape

(9476, 16)

In [21]:
data.to_csv("toyota.csv")